# Clase Práctica #5 (Compilación)

En esta clase estaremos adaptando el evaluador de expresiones aritméticas para trabajar sobre un _AST (Abstact Syntax Tree)_. Recordemos que el AST posee una estructura más cómoda para evaluar las reglas semánticas que el árbol de derivación. Además, evaluar en el AST en lugar que desde las reglas de la gramática atributada, simplifica significativamente la implementación de las reglas semánticas.

## Jerarquía del AST

Definamos una jerarquía de clases para los nodos del _AST_ del lenguaje de expresiones aritméticas. Utilizaremos las clases `Node` y `BinaryNode` como definiciones abstractas para agrupar y compactar la implementación de sus descendientes. Los nodos del AST serán exclusivamente instancias de `ConstantNumberNode`, `PlusNode`, `MinusNode`, `StarNode` y `DivNode`.

In [ ]:
class Node:
    def evaluate(self):
        raise NotImplementedError()

class ConstantNumberNode(Node):
    def __init__(self, lex):
        self.lex = lex
        self.value = float(lex)
        
    def evaluate(self):
        # Insert your code here!!!
        pass
        

class BinaryNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
    def evaluate(self):
        ## Insert your code here!!!
        # lvalue = ???
        # rvalue = ???
        return self.operate(lvalue, rvalue)
    
    @staticmethod
    def operate(lvalue, rvalue):
        raise NotImplementedError()
        

class PlusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        pass

class MinusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        pass

class StarNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        pass

class DivNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        # Insert your code here!!!
        pass


Veamos como luce una instancia concreta de un AST de expresiones. Nótese que la precedencia de los operadores debe seguir atrapada en el AST puesto que solo se desecharon los atributos sintácticos.

In [ ]:
from cmp.ast import get_printer
printer = get_printer(AtomicNode=ConstantNumberNode, BinaryNode=BinaryNode)

print(printer(
    PlusNode(
        MinusNode(
            ConstantNumberNode('5'),
            ConstantNumberNode('6')
        ), ConstantNumberNode('9')
    )
))

## Construcción del AST

Pasemos a definir la gramática del lenguaje de expresiones aritméticas junto con las reglas semánticas para formar el _AST_. Las reglas quedarán muy similares a las de la clase anterior, pero esta vez en lugar de operar los valores, construiremos el nodo del AST que denota la operación.

In [ ]:
from cmp.pycompiler import Grammar
from cmp.utils import pprint, inspect

G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y = G.NonTerminals('T F X Y')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) num')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
# E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]                    #
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
# X %= plus + T + X, None, None, None, None                               #
# X %= minus + T + X, None, None, None, None                              #
# X %= G.Epsilon, None                                                    #
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
# T %= F + Y, None, None, None                                            #
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
# Y %= star + F + Y, None, None, None, None                               #
# Y %= div + F + Y, None, None, None, None                                #
# Y %= G.Epsilon, None                                                    #
#                                                                         #
# ======================= { F --> num | ( E ) } ========================= #
# F %= num, None, None                                                    #
# F %= opar + E + cpar, None, None, None, None                            #
#                                                                         #
# ======================================================================= #
############################# END PRODUCTIONS #############################


Ensamblemos el pipeline de evaluación con los elementos que hemos ido implementando a lo largo de las pasadas clases.

Se realizará la siguiente cadena de transformaciones:
```
Entrada -> Tokens -> Parse Izquierdo -> AST -> Resultado
```    

In [ ]:
from cmp.utils import Token
from cmp.languages import BasicHulk
from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
from cmp.tools.evaluation import evaluate_parse

hulk = BasicHulk(G)
firsts = hulk.firsts
follows = hulk.follows
tokenize_text = hulk.tokenizer

M = build_parsing_table(G, firsts, follows)
parser = metodo_predictivo_no_recursivo(G, M)

def run_pipeline(text, value, parser, formatter):
    tokens = tokenize_text(text)
    pprint(tokens, '================Tokens================')
    left_parse = parser(tokens)
    pprint(left_parse, '==============Left-Parse==============')
    ast = evaluate_parse(left_parse, tokens)
    pprint(formatter(ast), '=================AST==================')
    result = ast.evaluate()
    pprint(f'{text} = {result}', '================Result================')
    assert result == value

Comprobemos que la asociatividad de los operadores no se perdió.

In [ ]:
run_pipeline('1 - 1 - 1', -1, parser, printer)

In [ ]:
run_pipeline('1 - ( 1 - 1 )', 1, parser, printer)

## Adicionando operador _potencia_

Añadamos el operador potencia al lenguaje. Para ello, realizaremos las modificaciones pertinentes a cada una de las fases del evaluador. No será necesario **copia y pegar** código de otras clases, ni modificar el código fuente del módulo `cmp` que se distribuye junto al _notebook_.

Usaremos el símbolo `^` para denotar al operador potencia. Este es un operador binario que computa $a^b$ siendo, `a` y `b` los operandos izquierdo y derecho respectivamente. Por ejemplo, `2 ^ 4` computa $2^4$. El operador potencia asocia hacia la derecha (contrario a los operadores: +, -, \* y /). Por tanto, `4 ^ 3 ^ 2` computa $4^{3^2} = 4^9$ en lugar de $(4^3)^2 = 4^6$.

In [ ]:
# An empty cell. Just in case you don't know how to create new ones


In [ ]:
# An empty cell. Just in case you don't know how to create new ones.


In [ ]:
# An empty cell. Just in case you don't know how to create new ones :p


Como ayuda se proveen los conjunto _First_ y _Follow_ precomputados de lo que consideramos la gramática **natural** a obtener. De igual forma se tiene la tabla y parser LL(1). Siéntase libre de utilizar el código siguiente pero puede reemplazarlo por sus propias implementaciones en caso de que no pueda (o no entienda) cómo utilizarlo.

In [ ]:
from cmp.languages import PowHulk
pow_hulk = PowHulk(G)

firsts = pow_hulk.firsts
follows = pow_hulk.follows


from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
M = build_parsing_table(G, firsts, follows)
parser = metodo_predictivo_no_recursivo(G, M)



**Comprobemos que el operador potencia asocia hacia la derecha.**

In [ ]:
run_pipeline('4 ^ 3 ^ 2', 262144, parser, printer)

**Comprobemos que tiene más precedencia que el resto de los operadores.**

In [ ]:
run_pipeline('2 * 3 ^ 4 + 1 * 5', 167, parser, printer)

**Comprobemos que puede subordinarse a otros operadores usando paréntesis.**

In [ ]:
run_pipeline('3 ^ ( 1 + 1 ) ^ 2', 81, parser, printer)

## Adicionando _declaraciones de variables_

Añadamos variables al lenguaje de expresiones para acercarnos más a `HULK`. Para ello, agregue la expresión `let-in` al lenguaje. Dicha expresión sigue la siguiente sintaxis:

```
let
    <declaration-list>
in
    <expr>
```
donde `<expr>` denota cualquier expresión del lenguaje (incluyendo el propio `let-in`) y `<declaration-list>` representa una secuencia de declaraciones de la forma `<id> = <expr>` separadas por "`,`".

El valor de evaluación de la expresión `let-in` es el valor de evaluación de `<expr>`.  
Las variables declaradas en `<declaration-list>` serán visibles a partir de su declaración pero únicamente dentro de la expresión `let-in` que las contiene (incluye `<expr>`). Si `<expr>` contiene a su vez una expresión `let-in`, la declaración de una variable con el mismo nombre que una en el `let-in` padre **ocultará** la del padre.  
Por ejemplo, la expresión:
```
let
    x = 1,
    y = 2
in
    3 + (let x = 4, z = 5 in x + y + z) + x
```
equivalente a:
```
let x = 1, y = 2 in 3 + (let x = 4, z = 5 in x + y + z) + x
```
evalúa `15`.

In [ ]:
# An empty cell. Just in case you don't know how to create new ones

In [ ]:
# An empty cell. Just in case you don't know how to create new ones

In [ ]:
# An empty cell. Just in case you don't know how to create new ones